# Music Generation using an LSTM

#### Final Project for Deep Learning (CS 7643)

By Daeil Cha, Daniel Dias, Chitwan Kaudan

### Global Variables

In [1]:
# data_path = "../../lmd_matched"
data_path = "../clean-data"
saved_models_path = "../saved-models"

num_epochs = 10 # 1000
batch_size = 10
num_time_steps = 256
num_total_songs = 500

### Environment

In [2]:
import os

import torch

import numpy as np
import matplotlib.pyplot as plt

from IPython.core.debugger import set_trace
from getdata import getBatch

%matplotlib inline

%load_ext autoreload
%autoreload 2

learning_rate = 1e-6

##### Pytorch GPU/CPU

In [3]:
# dtype = torch.FloatTensor
# device = torch.device("cpu")

dtype = torch.cuda.FloatTensor
device = torch.device("cuda:0")

print(device)

cuda:0


### Data

#### Load In Data

In [4]:
start = 0
all_data = []

while start < num_total_songs:
    batch = np.array(getBatch(start, batch_size, num_time_steps, data_path), dtype=float)
    all_data.append(batch)
    # Shape should be (batch_size x num_time_steps x note_range x pitch/articulation)
    start += batch_size

all_data = np.concatenate(all_data, axis=0)

In [5]:
all_data.dtype

dtype('float64')

#### Partition Data

In [6]:
x_train = None
y_train = None

x_val = None
y_val = None

x_test = None
y_test = None

all_expected = np.empty(all_data.shape)
all_expected[:, 0:all_expected.shape[1]-1] = all_data[:, 1:all_data.shape[1]]
all_expected[:, all_expected.shape[1]-1] = 0

all_data = np.reshape(all_data, (num_total_songs, num_time_steps, -1))
all_expected = np.reshape(all_expected, (num_total_songs, num_time_steps, -1))

print("all data:", all_data.shape)
print("all expected:", all_expected.shape)

orig_dataset = torch.utils.data.TensorDataset(torch.from_numpy(all_data), torch.from_numpy(all_expected))
x_train, x_test = torch.utils.data.random_split(orig_dataset, [450, 50])

x_train_loader = torch.utils.data.DataLoader(x_train, batch_size=batch_size, shuffle=True)
x_test_loader = torch.utils.data.DataLoader(x_test)

all data: (500, 256, 156)
all expected: (500, 256, 156)


## Functions

#### Train Steps

In [7]:
def train_step(x, y, model, loss_criterion, optimizer):
    y_pred = model(x)

    # Compute and print loss
    # loss = loss_criterion(torch.max(y_pred, dim=1).indices, y)
    loss = loss_criterion(y_pred, y)
    ret_val = loss.item()

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    return ret_val

def test_step(x, y, model, loss_criterion):
    predictions = model(x)
    loss = loss_criterion(y_pred, y)
    
    return loss.item()

#### Save/Load Model

In [8]:
def save_model(model, model_name):
    torch.save(model.state_dict(), os.path.join(saved_models_path, "/", model_name))

def load_model_parameters(model, model_name):
    model.load_state_dict(torch.load(os.path.join(saved_models_path, "/", model_name)))

def load_new_model(model_name, model_constructor, *args):
    model = model_constructor(args)
    load_model_parameters(model, model_name)
    return model

### Model

In [9]:
from model.main_model import MusicGeneration

model = MusicGeneration(time_sequence_len=num_time_steps, batch_size=batch_size, time_hidden_size=36)

loss_criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

#### Move To Correct Device

In [10]:
model.to(device)

MusicGeneration(
  (lstm_time0): LSTM(156, 36, batch_first=True)
  (lstm_note0): LSTM(9216, 156, batch_first=True)
  (dropout): Dropout(p=0.2, inplace=False)
)

## Training

In [11]:
for epoch in range(num_epochs):
    print("Epoch:", epoch)
    for i, data in enumerate(x_train_loader, 0):
        x, y = data[0].to(device), data[1].to(device)

        loss = train_step(x, y, model, loss_criterion, optimizer)

        if i % 100 == 0:
            print(" -", loss)

Epoch: 0


RuntimeError: cuDNN error: CUDNN_STATUS_BAD_PARAM

## Test

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        x, y = data[0].to(device), data[1].to(device)

        loss = test_step(x, y, model, loss_criterion)

print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

##### IPDB Test Code

In [ ]:
set_trace()

def test_code():
    set_trace()

    train_iter = iter(x_train_loader)
    data = next(train_iter)
    x, y = data[0].to(device), data[1].to(device)

    loss = train_step(x, y, model, loss_criterion, optimizer)

test_code()

--Return--
None
> <ipython-input-13-1d033bb601ef>(1)<module>()
----> 1 set_trace()
      2 
      3 def test_code():
      4     set_trace()
      5 

ipdb> c
> <ipython-input-13-1d033bb601ef>(6)test_code()
      4     set_trace()
      5 
----> 6     train_iter = iter(x_train_loader)
      7     data = next(train_iter)
      8     x, y = data[0].to(device), data[1].to(device)

ipdb> n
> <ipython-input-13-1d033bb601ef>(7)test_code()
      5 
      6     train_iter = iter(x_train_loader)
----> 7     data = next(train_iter)
      8     x, y = data[0].to(device), data[1].to(device)
      9 

ipdb> n
> <ipython-input-13-1d033bb601ef>(8)test_code()
      6     train_iter = iter(x_train_loader)
      7     data = next(train_iter)
----> 8     x, y = data[0].to(device), data[1].to(device)
      9 
     10     loss = train_step(x, y, model, loss_criterion, optimizer)

ipdb> n
> <ipython-input-13-1d033bb601ef>(10)test_code()
      8     x, y = data[0].to(device), data[1].to(device)
      9 
--->

ipdb> s
> c:\users\daniel\anaconda3\envs\cs7643music\lib\site-packages\torch\nn\modules\module.py(532)__call__()
    530             result = self._slow_forward(*input, **kwargs)
    531         else:
--> 532             result = self.forward(*input, **kwargs)
    533         for hook in self._forward_hooks.values():
    534             hook_result = hook(self, input, result)

ipdb> s
--Call--
> c:\users\daniel\documents\cs 7643\sp20\proj\makingmusic\src\model\main_model.py(44)forward()
     42         return self
     43 
---> 44     def forward(self, x):
     45         time_outs, hidden_time_n = self.lstm_time0(x, self.hidden_time0)
     46         note_outs, hidden_note_n = self.lstm_note0(time_outs, self.hidden_note0)

ipdb> s
> c:\users\daniel\documents\cs 7643\sp20\proj\makingmusic\src\model\main_model.py(45)forward()
     43 
     44     def forward(self, x):
---> 45         time_outs, hidden_time_n = self.lstm_time0(x, self.hidden_time0)
     46         note_outs, hidden_note_